In [4]:
# Resolution.py
# September 2019 
# Minor tweaks by BCB, DMH, TDR from code orignially developed by Armin @ CZ
# Runs on Python 3.7, uses 'exifread' instead of pyexiv2 for tiff metadata parsing, avoids use of intermediate text file for metadata.

from PIL import Image
from PIL import ImageOps
import numpy
import wx
import os
import sys
import math

# import pyexiv2 #commented out by B

import exifread

#import piexif

def image_import():
       pixelsize=0
       magnification=0
       #app = wx.App()
       #frame = wx.Frame(None, -1, "Open file")
       #dirname = ''
       #dlg = wx.FileDialog(frame, 'Open file', dirname, "","TIF files (*.tif)|*.tif|All files (*.*)|*.*")
       path = None

       #if dlg.ShowModal() == wx.ID_OK:
       #       filename=dlg.GetFilename()
       #       dirname=dlg.GetDirectory()
       #       dlg.Destroy()
       # path = os.path.join(dirname,filename)
       dirname = "." 
       filename = "image.tif"
       path = os.path.join(dirname,filename)
    
       if not path == None:
              image = Image.open(path)
              exifFile= open(str(path),'rb')
              tags = exifread.process_file(exifFile, strict=False)

            # SEMINFO is denoted by tag 'Image Tag 0x8546'
              if "Image Tag 0x8546" in tags.keys():
                      print("Found SEM Info")
                      stuff = tags["Image Tag 0x8546"]
                      stuffVal = stuff.values
                      # SEM INFO metadata is a very long binary string and includes data that seems to be non decodable ascii / utf-8.
                      # The info. we need is in the first 100 characters so ignore everything after that
                      stuffVal = stuffVal[0:100].decode('utf-8')
                      #print(stuffVal)
                      stuffValList = stuffVal.splitlines()
                      
                      #print (len(stuffValList))
                      #print(stuffValList[3:4])

                      # Our needed info is 4th and 5th items extracted
                      pixelsize = stuffValList[3]
                      magnification = stuffValList[4]
              else:
                      print("Failed to find SEM INFO, exiting")
                      sys.exit(0)
       else:
              sys.exit(0)
       return image,path,float(magnification),float(pixelsize)


def image_cropping(image):
        image=image.convert("L")

        im1h = image.crop((306,10,818,522))
        im2h = image.crop((256,10,768,522))
        im3h = image.crop((206,10,718,522))
        im4h = image.crop((306,60,818,572))
        im5h = image.crop((256,60,768,572))
        im6h = image.crop((206,60,718,572))

        im1v = im1h.rotate(90)
        im2v = im2h.rotate(90)
        im3v = im3h.rotate(90)
        im4v = im4h.rotate(90)
        im5v = im5h.rotate(90)
        im6v = im6h.rotate(90)

        im1v=ImageOps.autocontrast(im1v, cutoff=0)
        im2v=ImageOps.autocontrast(im2v, cutoff=0)
        im3v=ImageOps.autocontrast(im3v, cutoff=0)
        im4v=ImageOps.autocontrast(im4v, cutoff=0)
        im5v=ImageOps.autocontrast(im5v, cutoff=0)
        im6v=ImageOps.autocontrast(im6v, cutoff=0)
        
        width = im1v.size[0]
        height = im1v.size[1]
        return im1v,im2v,im3v,im4v,im5v,im6v,width,height

# Transform image into array #

def im_array(image,width):
       #LOOP = 6
       #print numpy.reshape(image.getdata(),(-1,width))
       return numpy.reshape(image.getdata(),(-1,width)) #return 6 image arrays (512x512)
       
# Fouriertransform algorithmens # 

def fft_on_array(array):
       x = numpy.arange(0,512,1)
       window = []
       for i in range(0,512,1):
                function =  0.5*(1 + math.cos(2*math.pi*(i-256)/512))    #von Hann Window#
                window.append(function)
       signal = []
       signal = numpy.multiply(array,window)
       
       #return abs(numpy.fft.fftpack.fftshift(numpy.fft.fft(signal)))
       return abs(numpy.fft.fftshift(numpy.fft.fft(signal)))

       #print abs(numpy.fft.fftpack.fftshift(numpy.fft.fft(signal)))
       #converted 2-d FFT array into 1-d(512)FFT array (512*6 lines altogether)

def fft_averaging(array,height):
        #LOOP = 6
        #print fft_on_array(array)  #6 FFT arrays (512x512)
        #fft_on_array[a] corresponds to 1-d FFT array (512) (6 times)
        fft_average = 0
        for a in range(0,height,1):
                fft_average = fft_average + fft_on_array(array[a])
        #print fft_average/height
        return fft_average/height #Average per image

def subimage_averaging(image1,image2,image3,image4,image5,image6):
        subimage_averaging = (image1+image2+image3+image4+image5+image6)/6
        return subimage_averaging

        
def noise(fft_average):
        noise1 = 0
        noise2 = 0
        noise3 = 0
        noise4 = 0
        noise5 = 0
        noise6 = 0
        noise7 = 0
        noise8 = 0
        noise9 = 0
        noise10 = 0
        #noiseSUM = 0
        
        for e in range(1,21,1):
                noise1 = noise1 + fft_average[e]
        noise1 = noise1/20

        for e in range(11,31,1):
                noise2 = noise2 + fft_average[e]
        noise2 = noise2/20

        for e in range(21,41,1):
               noise3 = noise3 + fft_average[e]
        noise3 = noise3/20

        for e in range(31,51,1):
               noise4 = noise4 + fft_average[e]
        noise4 = noise4/20

        for e in range(41,61,1):
               noise5 = noise5 + fft_average[e]
        noise5 = noise5/20

        for e in range(51,71,1):
               noise6 = noise6 + fft_average[e]
        noise6 = noise6/20

        for e in range(61,81,1):
               noise7 = noise7 + fft_average[e]
        noise7 = noise7/20

        for e in range(71,91,1):
               noise8 = noise8 + fft_average[e]
        noise8 = noise8/20

        for e in range(81,101,1):
               noise9 = noise9 + fft_average[e]
        noise9 = noise9/20

        for e in range(91,111,1):
               noise10 = noise10 + fft_average[e]
        noise10 = noise10/20

        #for e in range (1,111,1):
               #noiseSUM = noiseSUM+fft_average[e]
        #noiseSUM = noiseSUM/110
        
        noise_average = (noise1+noise2+noise3+noise4+noise5+noise6+noise7+noise8+noise9+noise10)/10
        #Standard deviation
        sdnoise =math.sqrt(((noise_average-noise1)**2+(noise_average-noise2)**2+(noise_average-noise3)**2+(noise_average-noise4)**2+(noise_average-noise5)**2+(noise_average-noise6)**2+(noise_average-noise7)**2+(noise_average-noise8)**2+(noise_average-noise9)**2+(noise_average-noise10)**2)/10)

        noise=min(noise1, noise2, noise3, noise4, noise5, noise6, noise7, noise8, noise9, noise10)
        #noise = noise_average
        
       # print
       # print ('noise1: ', noise1)
       # print ('noise2: ', noise2)
       # print ('noise3: ', noise3)
       # print ('noise4: ', noise4)
       # print ('noise5: ', noise5)
       # print ('noise6: ', noise6)
       # print ('noise7: ', noise7)
       # print ('noise8: ', noise8)
       # print ('noise9: ', noise9)
       # print ('noise10: ', noise10)
       # print ('noise: ', noise)
       # print ('SD noise: ', sdnoise)
       # print ('log noise:',numpy.log10(noise))
       # print()
                
        noise_array =[]
        for b in range(1,len(fft_average),1): 
                noise_array.append(noise)
        return noise,sdnoise,noise_array
                
        
def resolution(fft_average,Noise,mag,pixelsize,sdnoise):
        Parameter=0
        ParameterB=0
        FC=0 #Frequency Count
       
        #print ('in resolution', Noise)
        for c in range(256,len(fft_average),1):
                FC=FC+1 #Frequency Count
                if (fft_average[c]-Noise)<=0:
                        Parameter=Parameter+1 
                        if Parameter==1: #first time signal is below noise
                                Frequency_max_1=(c-256)*(512**(-1)) #as c goes from 1 to 512                               
                        if Parameter==2: #second time signal is below noise
                                Frequency_max_2=(c-256)*(512**(-1)) #as c goes from 1 to 512
                        if Parameter==3: #third time signal is below noise
                                Frequency_max_3=(c-256)*(512**(-1)) #as c goes from 1 to 512
                        if Parameter==4: #fourth time signal is below noise
                                Frequency_max_4=(c-256)*(512**(-1)) #as c goes from 1 to 512
                        if Parameter==5: #fifth time signal is below noise
                                Frequency_max_5=(c-256)*(512**(-1)) #as c goes from 1 to 512 
                        if Parameter<=5:print("Frequency Count:",FC)

       #Obtaining resolution from 10% above noise as comparison
                if (fft_average[c]-1.1*Noise)<=0: #Noise Level 10% higher up
       
                      ParameterB=ParameterB+1 #first time signal is below noise
                      if ParameterB==1:
                             Frequency_maxB=(c-256)*(512**(-1)) #as c goes from 1 to 512
                             
                                                     
        #pixelsize = (114.3296*1000000)/(1024*float(mag))
        pixelsize = float (pixelsize)*1000000000
        Frequency_max=0.2*(Frequency_max_1+Frequency_max_2+Frequency_max_3+Frequency_max_4+Frequency_max_5)
        Frequency_max=Frequency_max_1
        Res_frequency=(Frequency_max)**(-1)*pixelsize
        Res_frequencyB=(Frequency_maxB)**(-1)*pixelsize

        R50 = 0.832554611*0.5*Res_frequency  # Spot radius according to Rayleigh' s Criterion
        R0 = 0.5*Res_frequency
        Error=(abs(R0-R50))/R50 #Check edge disturbance
       # print ('Resolution N/k_max=:',Res_frequency)
       # print ('Frequency 1=:',Frequency_max_1)
       # print ('Frequency 2=:',Frequency_max_2)
       # print ('Frequency 3=:',Frequency_max_3)
       # print ('Frequency 4=:',Frequency_max_4)
       # print ('Frequency 5=:',Frequency_max_5)
       # print ('Frequency =:',Frequency_max)
       # print ('Pixelsize =:', pixelsize)
       # print ('Magnification =:', mag)

        print  (' R50 =:',R50)
       # print ('R0 =:',R0)
       # print ('Error =:',Error)

        R50_rounded = round (R50, 2)
        R50_string = str(R50_rounded)

        ### Save results in file ###
        with open("Results.txt",'w') as f:
               f.write("Resolution = ")
               f.write(R50_string)
               f.write (" nm")
               f.write("\n")
               f.write ("'Could add more info here'")

      
        return R50,Res_frequency,Frequency_max,R0,Error

def SNR(fft_average,Noise):
        PNoise = Noise * Noise * 512
        PSignal = 0
        for c in range(0,len(fft_average),1):
                PSignal = PSignal + ((fft_average[c])**2)
        #print ('PSignal: ' ,PSignal)
        #print ('PNoise: ' , PNoise)
        SNR = PSignal/PNoise
        SNR = math.sqrt(SNR)
       # print (' SNR: ', SNR)
       # print ("AAA",numpy.log10(PSignal)- numpy.log10(PNoise))
        #return SNR

def Contrast(image,SNR):
       image = image.crop((0,0,1024,500)) # get rid of picture information line #
       histogram = image.histogram()
       #print len(histogram)

       Parameter = 0
       Imin = 0
       Imax = 256
       
       for i in range(0,255,1):
              if histogram[i] > 10 :
                     Parameter = Parameter +1
                     if Parameter ==1: Imin = i
       Parameter = 0
                           
       for j in range(255,0,-1):
              if histogram[j] > 10 :
                     Parameter = Parameter + 1
                     if Parameter ==1: Imax = j
                  
       contrast = round((Imax - Imin)*(Imax + Imin)**(-1),2)
      # print ('Contrast: ',contrast)

       Contrast = 0.5*(contrast * 100)
       
       return Contrast

def error_message():
        class MyButtons(wx.Dialog):
                def __init__(self, parent, id, title):
                    wx.Dialog.__init__(self, parent, id, title, size=(300, 120))

                    self.quote = wx.StaticText(self, -1, 'Image can not be evaluated. Please contact R/D', wx.Point(20,20))
                                        
                    wx.Button(self, 1, 'Close program', (20, 60))
                    wx.Button(self,2 ,'New image',(130,60))
                    self.Bind(wx.EVT_BUTTON, self.OnClose, id=2)
                    self.Bind(wx.EVT_BUTTON, self.new, id=1)
                    self.ShowModal()
                    self.Destroy()
                    
                def OnClose(self, event):
                        self.Close(True)

                def new(self, event):
                        self.Close(True)
                        main()

        app = wx.App(3)
        MyButtons(None, -1, 'error message')
        app.MainLoop()

def read_mag():
    app = wx.App(1)
    dlg = wx.TextEntryDialog(None, 'Please enter magnification','Magnification')
    dlg.SetValue("111650")
    dlg.ShowModal()
    a = dlg.GetValue()
    return a

def main():
        delta_noise = 0
        image,path,mag,pixelsize = image_import()
        im1v,im2v,im3v,im4v,im5v,im6v,width,height = image_cropping(image)

        im1v_array = im_array(im1v,width)
        im2v_array = im_array(im2v,width)
        im3v_array = im_array(im3v,width)
        im4v_array = im_array(im4v,width)
        im5v_array = im_array(im5v,width)
        im6v_array = im_array(im6v,width)

        fft_im1v_average = fft_averaging(im1v_array,height)
        fft_im2v_average = fft_averaging(im2v_array,height)
        fft_im3v_average = fft_averaging(im3v_array,height)
        fft_im4v_average = fft_averaging(im4v_array,height)
        fft_im5v_average = fft_averaging(im5v_array,height)
        fft_im6v_average = fft_averaging(im6v_array,height)

        fft_average = subimage_averaging(fft_im1v_average,fft_im2v_average,fft_im3v_average,fft_im4v_average,fft_im5v_average,fft_im6v_average)

        Noise,sdnoise,noise_array = noise(fft_average)
        SNRatio = SNR(fft_average,Noise)
        R50,Res_frequency,Frequency_max,R0,Error = resolution(fft_average,Noise,mag,pixelsize,delta_noise)
        contrast = Contrast(image,SNRatio)

        #if Error<=0.3: #don' t allow more than 30% deviation between R50 and R0
        #output_window(R50,SNRatio,path,mag,contrast,Frequency_max,Res_frequency,R0)
        #else: error_message()
        #print numpy.reshape(image.getdata(),(-1,width))

                
#Main program#
        
main()


Possibly corrupted field Tag 0x8546 in Image IFD
Found SEM Info
Frequency Count: 123
Frequency Count: 132
Frequency Count: 153
Frequency Count: 154
Frequency Count: 157
 R50 =: 23.082899239007855
